In [1]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import matplotlib.pyplot as plt
import numpy as np

# Load data

In [2]:
dl_train_all, dl_val_all, dl_test_all = load_all_data(normalize=True)

# Train / Test

## Params

In [3]:
le_net = LeNet()

# Can use any network with ten input and two output nodes here
tail_net = nn.Linear(20, 2)

nb_epochs = 25
n_cv = 10 
n_cv_weight = 5

## Hyperparam opt the weight 
- Can skip and use result if using same architecture

In [4]:
weights = np.linspace(0.1, 1, 10)

In [5]:
weight_test_accuracies = np.zeros((n_cv_weight, len(weights)))

In [6]:
def hyperparam_opt(weights, n_cv_weight=5):
    for j, weight in enumerate(weights): 
    # 5 fold cross validate here 
        for i in range(n_cv_weight):
            siamese_le_net = Siamese(le_net, 
                                     softmax=True,
                                     target=tail_net, 
                                     weight_aux=weight)
            trainer = Trainer(nb_epochs=nb_epochs)
            trainer.fit(siamese_le_net, dl_train_all, dl_val_all, verbose=False)
            acc_test = trainer.test(siamese_le_net, dl_test_all, test_verbose=True, return_acc=True)
            weight_test_accuracies[i,j] = acc_test
    avg_weight_test_accuracies = np.mean(weight_test_accuracies, axis=0)
    best_weight = weights[np.argmax(avg_weight_test_accuracies)]
    return best_weight

In [7]:
best_weight = hyperparam_opt(weights, n_cv_weight=5)

/home/puck/coding/EE-559-Team/Project1/src/models.py:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


loss_test=0.42	 acc_test=89.06
loss_test=0.3	 acc_test=90.92
loss_test=0.27	 acc_test=91.21
loss_test=0.27	 acc_test=91.5
loss_test=0.33	 acc_test=89.26
loss_test=0.33	 acc_test=91.21
loss_test=0.35	 acc_test=91.21
loss_test=0.46	 acc_test=90.33
loss_test=0.44	 acc_test=91.11
loss_test=0.49	 acc_test=90.04
loss_test=0.5	 acc_test=91.02
loss_test=0.56	 acc_test=90.72
loss_test=0.57	 acc_test=90.72
loss_test=0.59	 acc_test=90.53
loss_test=0.61	 acc_test=90.72
loss_test=0.63	 acc_test=90.92
loss_test=0.69	 acc_test=90.72
loss_test=0.64	 acc_test=91.02
loss_test=0.79	 acc_test=89.75
loss_test=0.77	 acc_test=89.94
loss_test=0.7	 acc_test=90.62
loss_test=0.81	 acc_test=90.23
loss_test=0.88	 acc_test=90.14
loss_test=0.95	 acc_test=89.06
loss_test=0.9	 acc_test=90.62
loss_test=1.08	 acc_test=89.26
loss_test=1.04	 acc_test=89.75
loss_test=0.81	 acc_test=91.5
loss_test=0.99	 acc_test=90.33
loss_test=1.05	 acc_test=90.33
loss_test=1.09	 acc_test=90.33
loss_test=1.07	 acc_test=90.53
loss_test=1.17

In [8]:
# keep best weight
best_weight

0.2

In [9]:
weight_aux = best_weight

In [10]:
test_accuracies = []

In [11]:
for i in range(n_cv):
    siamese_le_net = Siamese(le_net, weight_aux=weight_aux)
    trainer = Trainer(nb_epochs=nb_epochs)
    trainer.fit(siamese_le_net, dl_train_all, dl_val_all, verbose=False)
    acc_test = trainer.test(siamese_le_net, dl_test_all, test_verbose=True, return_acc=True)
    test_accuracies.append(acc_test)

loss_test=0.43	 acc_test=89.75
loss_test=0.31	 acc_test=90.04
loss_test=0.31	 acc_test=89.84
loss_test=0.32	 acc_test=90.14
loss_test=0.38	 acc_test=90.14
loss_test=0.43	 acc_test=90.43
loss_test=0.43	 acc_test=91.02
loss_test=0.5	 acc_test=91.02
loss_test=0.58	 acc_test=90.33
loss_test=0.62	 acc_test=90.62


In [12]:
avg_test_acc = np.mean(test_accuracies)
print('avg test acc', avg_test_acc)
stdev_test_acc = np.std(test_accuracies)
print('stdev test acc', stdev_test_acc)

avg test acc 90.33300000000001
stdev test acc 0.42178311962429055


In [13]:
from src.utils import print_param_count

In [14]:
print_param_count(siamese_le_net)

Total number of parameters:     72268
Number of trainable parameters: 72268
